In [ ]:
import pickle as pkl
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
import cv2

In [ ]:
feature_list = np.array(pkl.load(open('feature_list.pkl','rb')))
filenames = pkl.load(open('filenames.pkl','rb'))

In [ ]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

# Create a Sequential model and add GlobalMaxPooling2D layer
seq_model = Sequential([model, GlobalMaxPooling2D()])
print(seq_model.summary())

In [ ]:
img = image.load_img('sample/jersey.jpg', target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

In [ ]:
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
neighbors.fit(feature_list)

In [ ]:
distances, indices = neighbors.kneighbors([normalized_result])
print(indices)

In [ ]:
for file in indices[0]:
    temp_img = cv2.imread(filenames[file])
    cv2.imshow('output',cv2.resize(temp_img, (512, 512)))
    cv2.waitKey(0)

cv2.destroyAllWindows()